In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
# https://lol.fandom.com/wiki/LEC/2022_Season/Spring_Season/Match_History

### Request match data

In [3]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36"}
html = requests.get("https://gol.gg/game/stats/44359/page-summary/", headers=headers).content
soup = BeautifulSoup(html, 'html.parser')

In [4]:
# print(soup.prettify())

# How find divs in parser
# https://linuxpip.org/beautifulsoup-find-an-element-by-class/

### Name of teams in the match

In [5]:
#Team blue side
soup.find("div", class_="row rowbreak fond-main-cadre p-4").find_all("div", class_="col-4 col-sm-5 text-center")[0].get_text()

#Team red side
soup.find("div", class_="row rowbreak fond-main-cadre p-4").find_all("div", class_="col-4 col-sm-5 text-center")[1].get_text()

'CTBC Flying Oyster'

### Champions Banned and Picked

In [6]:
#First 5 is the banned champions for blue side
#Last 5 picke champions for blue side
blue_champs = soup.find("div", class_="row rowbreak fond-main-cadre p-4").find_all("div", class_="col-4 col-sm-5 text-center")[2].find_all("img")

#Same for the red side
red_champs = soup.find("div", class_="row rowbreak fond-main-cadre p-4").find_all("div", class_="col-4 col-sm-5 text-center")[3].find_all("img")

In [27]:
lst_blue_champs = []
for champ in blue_champs:
    lst_blue_champs.append(champ["src"].split('/')[-1].split('.')[0])

lst_red_champs = []
for champ in red_champs:
    lst_red_champs.append(champ["src"].split('/')[-1].split('.')[0])

In [28]:
# picks and bans of blue side
blue_banned = lst_blue_champs[:5]
blue_piked = lst_blue_champs[5:]

In [29]:
#picks and bans of red side
red_banned = lst_red_champs[:5]
red_piked = lst_red_champs[5:]

### Team Stats (Kills, Towers, Dragons and Barons)

In [10]:
#Ordem of prints, Kills, Towers, Dragons, Barons

#For blue team
for i in soup.find_all("div", class_="col-12 col-sm-6 pb-4")[0].find_all("span", class_="score-box"):
    #Keep only numbers in the string
    print(int(''.join(filter(str.isdigit, i.get_text()))))

#For red team
for i in soup.find_all("div", class_="col-12 col-sm-6 pb-4")[1].find_all("span", class_="score-box"):
    #Keep only numbers in the string
    print(int(''.join(filter(str.isdigit, i.get_text()))))

25
7
4
1
10
4
1
1


### KDA of teams

In [11]:
#Find all tables and get the first(blue side)
# soup.find_all('table')[1]

In [12]:
#Get the KDA of each team
table = soup.find_all("table", "table_list trhover")[1]
row_marker = 0
table_data = []

#lopp for each row in the table
for row in table.find_all('tr'):
    column_marker = 0
    column_data = []
    columns = row.find_all('td')

    #extract info for each collum and save in the list
    for column in columns:
        column_data.append(column.get_text())
    table_data.append(column_data)

In [13]:
table_data = table_data[1:]
table_data

[['Rest', '3/3/2 (1.7)', '8.4', '570', '0.31'],
 ['Gemini', '0/4/8 (2.0)', '4.6', '246', '0.71'],
 ['Mission', '4/3/3 (2.3)', '8.7', '731', '0.43'],
 ['Shunn', '1/7/5 (0.9)', '8.6', '382', '0.51'],
 ['Koala', '2/8/4 (0.8)', '1.2', '213', '3.00']]

In [14]:
df_kda = pd.DataFrame(table_data,columns=['player', 'kda', 'csM', 'dpM','wpM'])
df_kda

,player,kda,csM,dpM,wpM
0,Rest,3/3/2 (1.7),8.4,570,0.31
1,Gemini,0/4/8 (2.0),4.6,246,0.71
2,Mission,4/3/3 (2.3),8.7,731,0.43
3,Shunn,1/7/5 (0.9),8.6,382,0.51
4,Koala,2/8/4 (0.8),1.2,213,3.00


In [15]:
def player_kda(row):
    kills, deaths, other = row.split('/')
    assists = other.split(' ')[0]
    if int(deaths) > 0:
        kda = (int(kills) + int(assists))/int(deaths)
    else:
        kda = (int(kills) + int(assists))
        
    return round(kda, 2)


In [16]:
df_kda['kda'] = df_kda.apply(lambda x: player_kda(x['kda']),axis=1)

In [17]:
df_kda

,player,kda,csM,dpM,wpM
0,Rest,1.67,8.4,570,0.31
1,Gemini,2.00,4.6,246,0.71
2,Mission,2.33,8.7,731,0.43
3,Shunn,0.86,8.6,382,0.51
4,Koala,0.75,1.2,213,3.00
